In [2]:
import torch
import torch.nn.functional as F
from utils import load_our_sae, load_concatenated_sae, import_connor_sae

In [12]:
# load a single SAE
head_saes = {}
for hook_point_head_index in [0, 1]:
    model, sae, activations_loader = load_our_sae(hook_point_head_index=hook_point_head_index)
    head_saes[hook_point_head_index] = sae

Loading SAE for head # 0
Loaded pretrained model gelu-2l into HookedTransformer
Moving model to device:  cpu


Resolving data files:   0%|          | 0/28 [00:00<?, ?it/s]

Loading dataset shards:   0%|          | 0/24 [00:00<?, ?it/s]

Loading SAE for head # 1
Loaded pretrained model gelu-2l into HookedTransformer
Moving model to device:  cpu


Resolving data files:   0%|          | 0/28 [00:00<?, ?it/s]

Loading dataset shards:   0%|          | 0/24 [00:00<?, ?it/s]

In [30]:
%%capture
model, our_concatenated_sae, activations_loader = load_concatenated_sae()
connor_sae = import_connor_sae()

Loading SAE for head # 0
Loaded pretrained model gelu-2l into HookedTransformer
Moving model to device:  cpu


Resolving data files:   0%|          | 0/28 [00:00<?, ?it/s]

Loading dataset shards:   0%|          | 0/24 [00:00<?, ?it/s]

Loading SAE for head # 1
Loaded pretrained model gelu-2l into HookedTransformer
Moving model to device:  cpu


Resolving data files:   0%|          | 0/28 [00:00<?, ?it/s]

...

Loading dataset shards:   0%|          | 0/24 [00:00<?, ?it/s]

Loading SAE for head # 7
Loaded pretrained model gelu-2l into HookedTransformer
Moving model to device:  cpu


Resolving data files:   0%|          | 0/28 [00:00<?, ?it/s]

Loading dataset shards:   0%|          | 0/24 [00:00<?, ?it/s]

Concatenating all SAEs from individual attention heads


## Getting feature activations

In [4]:
# get model cache
batch_tokens = activations_loader.get_batch_tokens(batch_size=1)
B, T = batch_tokens.shape
_, cache = model.run_with_cache(batch_tokens, prepend_bos=True)

In [13]:
## forward pass through SAEs
# forward pass outputs (sae_out, feature_acts, loss, mse_loss, l1_loss, ghost_grad_loss)
feature_acts = {}
for head_id, sae in head_saes.items():
    head_feature_acts = sae(
        cache[sae.cfg.hook_point][:, :, hook_point_head_index, :]
    )[1] # (B, T, n_latents)
    feature_acts[head_id] = head_feature_acts

In [33]:
# Connor's SAE
hook_point = connor_sae.cfg.hook_point
connor_feature_acts = connor_sae(
    cache[hook_point].view(B, T, -1)
)[1]

In [34]:
# our concatenated SAE
hook_point = our_concatenated_sae.cfg.hook_point
our_concatenated_feature_acts = our_concatenated_sae(
    cache[hook_point].view(B, T, -1)
)[1]

### Normalizing feature activations

dict_items([(0, tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]])), (1, tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]]))])